#Iris Dataset Classification

This classification problem deals with the three species of the Iris flower and classifying them on the basis of their dimentions of their sepals and petals.

##Importing the important methods and libraries

In [ ]:
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np

##Importing the Dataset
Dataset link: https://www.kaggle.com/uciml/iris

In [ ]:
data_set = pd.read_csv(r"/content/drive/My Drive/datasets_19_420_Iris.csv") #The dataset is stored
data_set.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


##Obtaining the features and targets from the dataset
In this step, we obtain the features and the targets from the dataset.The target is encoded numerically. The features and targets are splitted into training data and testing data. The training target is now one hot encoded.

In [ ]:
target2 = data_set['Species'].values
features = data_set.drop(labels = ['Species','Id'],axis = 1).values

target1 = []
for s in target2:                       #Encoding the target strings to numbers
    if s == 'Iris-setosa':
        target1.append(0)
    elif s == 'Iris-versicolor':
        target1.append(1)
    else:
        target1.append(2)

target0 = np.array(target1)

X_train, X_test, Y_train, Y_test = train_test_split(features,target0,test_size = 0.2,stratify = target0,random_state = 20)  #Splitting the training and testing data
Y_train = keras.utils.to_categorical(Y_train,num_classes = 3)  #One hot encoding the training target

##Normalizing the training and testing features
There is a need to scale the features of the dataset so that all the features contribute to the prediction equally. We use Standard scaler from sklearn.preprocessing library. The scaler fits the training data for appropriate normalization and then transform both training and testing data by the same scaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Defining the Neural Network
This neural network has an input layer with 150 nodes and two hiddin layers with 100 and 50 nodes respectively. The activation function used for the input and the hidden layers is relu and for the output layer, we use softmax function for the probabilities. We need to classify the features into anyone of the three species and therefore we need three nodes in the output layer.

In [ ]:
model = Sequential()

model.add(Dense(150,activation = 'relu',input_shape = (features.shape[1],)))
model.add(Dense(100,activation = 'relu'))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(3,activation = 'softmax'))     #Output Layer

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 150)               750       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 153       
Total params: 21,053
Trainable params: 21,053
Non-trainable params: 0
_________________________________________________________________


#Compiling the Model
The model is now compiled by using the adam optimizer and categorical crossentropy loss. We want to display the accuracy while training, so we mention the accuracy metrics here itself.

In [ ]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

##Training the model
The model is trained for 20 epochs

In [ ]:
model.fit(X_train,Y_train,epochs = 20)

Epoch 1/20
4/4 [==============================] - 0s 3ms/step - loss: 1.0560 - accuracy: 0.4583
Epoch 2/20
4/4 [==============================] - 0s 3ms/step - loss: 0.8872 - accuracy: 0.7833
Epoch 3/20
4/4 [==============================] - 0s 3ms/step - loss: 0.7560 - accuracy: 0.8167
Epoch 4/20
4/4 [==============================] - 0s 2ms/step - loss: 0.6431 - accuracy: 0.8167
Epoch 5/20
4/4 [==============================] - 0s 2ms/step - loss: 0.5439 - accuracy: 0.8167
Epoch 6/20
4/4 [==============================] - 0s 2ms/step - loss: 0.4584 - accuracy: 0.8250
Epoch 7/20
4/4 [==============================] - 0s 2ms/step - loss: 0.3943 - accuracy: 0.8333
Epoch 8/20
4/4 [==============================] - 0s 2ms/step - loss: 0.3458 - accuracy: 0.8583
Epoch 9/20
4/4 [==============================] - 0s 2ms/step - loss: 0.3113 - accuracy: 0.8667
Epoch 10/20
4/4 [==============================] - 0s 3ms/step - loss: 0.2788 - accuracy: 0.8833
Epoch 11/20
4/4 [======================

##Obtaining the results
Saved the model for further usage. The predicions are obtained and stored in the 'result' array for the testing set.

In [ ]:
model.save("/content/drive/My Drive/iris_model.h5")
result = model.predict(X_test)

##Result processing
There is a need to process the obtained results. Since the output of the neural network is a set of probabilities for the different classes, we need to translate the predicted probabilities to their respective meanings. The information is now decoded for furthur understanding in human language.

In [ ]:
mapping = {0 : 'Iris-setosa',1 : 'Iris-versicolor',2 : 'Iris-virginica'}

predictions = []
actual = []
comments = []
test_case = []
for a in result:
    for i in range(len(a)):
        if a[i] == max(a):
            predictions.append(mapping[i])
count = 1
for i in Y_test:
    actual.append(mapping[i])
    test_case.append(count)
    count = count + 1

for i in range(len(predictions)):
    if predictions[i] == actual[i]:
        comments.append('Correct prediction')
    else:
        comments.append('Incorrect prediction')
predictions = np.array(predictions)
actual = np.array(actual)
comments = np.array(comments)
test_case = np.array(test_case)

#Tabular Representation of Predictions
We now represent the predictions of the neural networks along with their respective actual values and draw conclutions in a tabular form

In [ ]:
data = {"Test Case": test_case,"Actual Species" : actual,"Prediction" : predictions,"Comments" : comments}

result_table = pd.DataFrame(data,columns = ['Test Case','Actual Species','Prediction','Comments'])
result_table.head(len(actual))

,Test Case,Actual Species,Prediction,Comments
0,1,Iris-versicolor,Iris-versicolor,Correct prediction
1,2,Iris-versicolor,Iris-versicolor,Correct prediction
2,3,Iris-versicolor,Iris-versicolor,Correct prediction
3,4,Iris-virginica,Iris-virginica,Correct prediction
4,5,Iris-setosa,Iris-setosa,Correct prediction
5,6,Iris-setosa,Iris-setosa,Correct prediction
6,7,Iris-versicolor,Iris-versicolor,Correct prediction
7,8,Iris-versicolor,Iris-versicolor,Correct prediction
8,9,Iris-virginica,Iris-virginica,Correct prediction
9,10,Iris-versicolor,Iris-versicolor,Correct prediction


#THANK YOU
Made by: Neeraj Sreedhar(NIT GOA) \\